In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "meta-llama/Llama-3.2-1B"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
model = model.to("cuda")

In [35]:
raw_prompt = """You are an expert in multiple-choice questions. Your task is to select the best answer from the given options based on the provided context.

Context:
lets to it

Question: What were the key findings regarding the relationship between tumor size and hearing loss in patients with acoustic neuromas, as well as the effectiveness of the Metz test in assessing hearing loss?

Options:
A. There was no correlation between tumor size and hearing loss, and the Metz test was found to be ineffective in assessing hearing loss.
B. In a study of 125 patients with surgically confirmed acoustic neuromas, a significant correlation was found between tumor size and the degree of hearing loss, with no patients exhibiting normal hearing and the Metz test demonstrating the highest validity among various audiological assessments.
C. All patients exhibited normal hearing regardless of tumor size, and the Metz test was the least valid among audiological assessments.
D. A small tumor size was associated with severe hearing loss, and the Metz test was only moderately effective in evaluating hearing loss.

between A, B, C and D the best option is"""

In [36]:
raw_prompt

'You are an expert in multiple-choice questions. Your task is to select the best answer from the given options based on the provided context.\n\nContext:\nlets to it\n\nQuestion: What were the key findings regarding the relationship between tumor size and hearing loss in patients with acoustic neuromas, as well as the effectiveness of the Metz test in assessing hearing loss?\n\nOptions:\nA. There was no correlation between tumor size and hearing loss, and the Metz test was found to be ineffective in assessing hearing loss.\nB. In a study of 125 patients with surgically confirmed acoustic neuromas, a significant correlation was found between tumor size and the degree of hearing loss, with no patients exhibiting normal hearing and the Metz test demonstrating the highest validity among various audiological assessments.\nC. All patients exhibited normal hearing regardless of tumor size, and the Metz test was the least valid among audiological assessments.\nD. A small tumor size was associa

In [37]:
tokenizer.pad_token = tokenizer.eos_token
tokenized = tokenizer(
    raw_prompt,
    return_tensors="pt",
).to("cuda")

tokenized

{'input_ids': tensor([[128000,   2675,    527,    459,   6335,    304,   5361,  63726,   4860,
             13,   4718,   3465,    374,    311,   3373,    279,   1888,   4320,
            505,    279,   2728,   2671,   3196,    389,    279,   3984,   2317,
            382,   2014,    512,  10145,    311,    433,    271,  14924,     25,
           3639,   1051,    279,   1401,  14955,   9002,    279,   5133,   1990,
          36254,   1404,    323,  11011,   4814,    304,   6978,    449,  45166,
          21143,    442,    300,     11,    439,   1664,    439,    279,  27375,
            315,    279,   6344,     89,   1296,    304,  47614,  11011,   4814,
           1980,   3883,    512,     32,     13,   2684,    574,    912,  26670,
           1990,  36254,   1404,    323,  11011,   4814,     11,    323,    279,
           6344,     89,   1296,    574,   1766,    311,    387,  55288,    304,
          47614,  11011,   4814,    627,     33,     13,    763,    264,   4007,
            31

In [38]:
tokenizer.decode(tokenized["input_ids"][0])

'<|begin_of_text|>You are an expert in multiple-choice questions. Your task is to select the best answer from the given options based on the provided context.\n\nContext:\nlets to it\n\nQuestion: What were the key findings regarding the relationship between tumor size and hearing loss in patients with acoustic neuromas, as well as the effectiveness of the Metz test in assessing hearing loss?\n\nOptions:\nA. There was no correlation between tumor size and hearing loss, and the Metz test was found to be ineffective in assessing hearing loss.\nB. In a study of 125 patients with surgically confirmed acoustic neuromas, a significant correlation was found between tumor size and the degree of hearing loss, with no patients exhibiting normal hearing and the Metz test demonstrating the highest validity among various audiological assessments.\nC. All patients exhibited normal hearing regardless of tumor size, and the Metz test was the least valid among audiological assessments.\nD. A small tumor

In [39]:
with torch.no_grad():
    model.eval()
    # Generate the output
    out = model(
        **tokenized,
    )


In [40]:
out[0][-1]

tensor([[ 7.0544,  9.0268, 13.3232,  ..., -3.7595, -3.7596, -3.7596],
        [12.8574, 11.2714,  9.1116,  ...,  0.0396,  0.0400,  0.0398],
        [10.5716,  5.0304,  6.1321,  ..., -0.3093, -0.3089, -0.3093],
        ...,
        [13.4677, 12.2019, 10.0472,  ...,  1.5700,  1.5696,  1.5700],
        [12.9664, 14.3813, 10.0968,  ...,  1.2152,  1.2147,  1.2153],
        [ 4.0033,  4.3279,  5.4732,  ...,  2.9455,  2.9452,  2.9455]],
       device='cuda:0')

In [41]:
id = torch.argmax(out.logits[0][-1])

In [42]:
id

tensor(362, device='cuda:0')

In [43]:
tokenizer.decode(id)

' A'

In [44]:
generate = model.generate(
    input_ids=tokenized["input_ids"],
    attention_mask=tokenized["attention_mask"],
    do_sample=True,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [45]:
tokenizer.decode(generate[0])

'<|begin_of_text|>You are an expert in multiple-choice questions. Your task is to select the best answer from the given options based on the provided context.\n\nContext:\nlets to it\n\nQuestion: What were the key findings regarding the relationship between tumor size and hearing loss in patients with acoustic neuromas, as well as the effectiveness of the Metz test in assessing hearing loss?\n\nOptions:\nA. There was no correlation between tumor size and hearing loss, and the Metz test was found to be ineffective in assessing hearing loss.\nB. In a study of 125 patients with surgically confirmed acoustic neuromas, a significant correlation was found between tumor size and the degree of hearing loss, with no patients exhibiting normal hearing and the Metz test demonstrating the highest validity among various audiological assessments.\nC. All patients exhibited normal hearing regardless of tumor size, and the Metz test was the least valid among audiological assessments.\nD. A small tumor